**1)Розрахувати: Кількість покупців, суму здійснених покупок, середній чек та середню величину покупки на 1 клієнта, за додаткових умов**

*А. Враховувати покупки клієнтів, які стали клієнтами мережі із 2019 по 2022 рік, включно*

In [ ]:
SELECT DISTINCT Card_code
FROM Client
WHERE StartDate BETWEEN '2019-01-01' AND '2022-12-31';


*Б. Не брати до уваги покупки клієнтів, які здійснили за цей період мене 3-х покупок (випадкові клієнти)*

In [ ]:
SELECT DISTINCT Card_code
FROM Check
GROUP BY Card_code
HAVING COUNT(DISTINCT Check_id) >= 3;


*В. Не брати до уваги покупок клієнтів, в середній корзині у яких менше 5 СКЮ (шукачі мега-знижок, випадкові/імпульсні покупки)*

In [ ]:
SELECT DISTINCT Card_code
FROM Check
GROUP BY Card_code
HAVING AVG(DISTINCT Tovar_id) >= 5;


*Г. Враховувати в сумі покупок тільки товари підкатегорії FMCG (не входять САП-групи U та  V )*

In [ ]:
SELECT Sum
FROM Check
WHERE Tovar_id IN (
    SELECT Tovar_id
    FROM Product
    WHERE SAP_id NOT IN ('U', 'V'));

*А,Б,В,Г)Кількість покупців, суму здійснених покупок, середній чек та середню величину покупки на 1 клієнта*

In [ ]:
SELECT SUM(Sum) AS TotalSales,
       AVG(Sum) AS AvgPurchasePerCustomer
FROM Check
WHERE Card_code IN (
        SELECT Card_code
        FROM Client
        WHERE StartDate BETWEEN '2019-01-01' AND '2022-12-31'
        GROUP BY Card_code
        HAVING COUNT(DISTINCT Check_id) >= 3
            AND AVG(Tovar_id) >= 5)
    AND Card_code IN (
        SELECT DISTINCT Card_code
        FROM Check
        WHERE Tovar_id IN (
                SELECT Tovar_id
                FROM Product
                WHERE SAP_id NOT IN ('U', 'V')));

**2) Скласти Рейтинг-ТОП СКЮ за частотою потрапляння до кошику з 10-ти найбільш часто вживаних товарів.**

In [ ]:
WITH TopProducts AS (
    SELECT Tovar_id, SUM(Sum) AS TotalPurchase
    FROM Check
    GROUP BY Tovar_id
    ORDER BY TotalPurchase DESC LIMIT 10),
CustomerPurchase AS (
    SELECT Card_code, Tovar_id, 
    SUM(sum) AS TotalPurchase,
    RANK() OVER (PARTITION BY Card_code ORDER BY SUM(Sum) DESC) AS PurchaseRank 
    FROM Check
    GROUP BY Card_code, Tovar_id),
SELECT Tovar_id, 
    COUNT(*) AS CustomerCount,
    CAST(COUNT(*) AS FLOAT) / 1000 AS Rating
FROM CustomerPurchase
WHERE Tovar_id IN (SELECT Tovar_id FROM TopProduct) AND PurchaseRank <= 10
GROUP BY Tovar_id
ORDER BY Rating DESC LIMIT 50;


